In [1]:
import os
os.chdir('../../')
import sys
sys.path.insert(0, './python')
sys.path.append('/local-scratch/xca64/tmp/caffe-master/python/myFunc')
import caffe
import numpy as np
from pylab import *
%matplotlib inline

In [2]:
niter = 200
# losses will also be stored in the log
train_loss = np.zeros(niter)
scratch_train_loss = np.zeros(niter)

caffe.set_device(0)
caffe.set_mode_gpu()

In [37]:
solver = caffe.SGDSolver('models/color_constancy/solver.prototxt')
solver.net.copy_from('models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel')

In [26]:
import tempfile

In [31]:
def run_solver(solver, niter, disp_interval):
    blobs = ('loss', 'acc')
    loss, acc = (np.zeros(niter), np.zeros(niter))
    for it in range(niter):
        solver.step(1)  # run a single SGD step in Caffe
        loss[it] = (solver.net.blobs['loss'].data.copy())
        acc[it] = 0#(solver.net.blobs['loss_ang'].data.copy())
        if it % disp_interval == 0 or it + 1 == niter:
            loss_disp = 'loss: %.3f'%loss[it]
            print '%3d) %s Angular Erro %.3f' % (it, loss_disp, acc[it])     
            #print(solver.net.blobs['fc8_flickr'].data[1], solver.net.blobs['illu'].data[1])
    # Save the learned weights from both nets.
    weight_dir = tempfile.mkdtemp()
    name = 'firstTry'
    weights = {}
    filename = 'weights.%s.caffemodel' % name
    
    weights[name] = os.path.join(weight_dir, filename)
    solver.net.save(weights[name])
    
    return loss, acc, weights

In [38]:
loss_1, acc_1, weights_1 = run_solver(solver, 1000,10)

  0) loss: 45.797 Angular Erro 0.000
 10) loss: 14.247 Angular Erro 0.000
 20) loss: 10.157 Angular Erro 0.000
 30) loss: 7.536 Angular Erro 0.000
 40) loss: 7.600 Angular Erro 0.000
 50) loss: 6.920 Angular Erro 0.000
 60) loss: 6.690 Angular Erro 0.000
 70) loss: 6.781 Angular Erro 0.000
 80) loss: 5.732 Angular Erro 0.000
 90) loss: 6.745 Angular Erro 0.000
100) loss: 6.561 Angular Erro 0.000
110) loss: 7.540 Angular Erro 0.000
120) loss: 6.053 Angular Erro 0.000
130) loss: 5.353 Angular Erro 0.000
140) loss: 4.901 Angular Erro 0.000
150) loss: 5.053 Angular Erro 0.000
160) loss: 5.369 Angular Erro 0.000
170) loss: 4.482 Angular Erro 0.000
180) loss: 17.305 Angular Erro 0.000
190) loss: 20.603 Angular Erro 0.000
200) loss: 12.281 Angular Erro 0.000
210) loss: 15.472 Angular Erro 0.000
220) loss: 15.167 Angular Erro 0.000
230) loss: 8.961 Angular Erro 0.000
240) loss: 6.130 Angular Erro 0.000
250) loss: 13.526 Angular Erro 0.000
260) loss: 16.074 Angular Erro 0.000
270) loss: 11.363 

In [ ]:
print [solver.net.blobs['fc8_flickr'].data, solver.net.blobs['illu'].data]


In [ ]:
solver = caffe.SGDSolver('models/color_constancy/solver.prototxt')
solver.net.copy_from('models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel')

In [ ]:
solver.step(1)  # SGD by Caffe

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [ ]:
def deprocess_net_image(image, gamma):
    image = image.copy()              # don't modify destructively
    image = image[::-1]               # BGR -> RGB
    image = image.transpose(1, 2, 0)  # CHW -> HWC
    image += [123, 117, 104]          # (approximately) undo mean subtraction

    # clamp values in [0, 255]
    image[image < 0], image[image > 255] = 0, 255
    
    image = image.astype(np.float32)
    image = image/255
    image = image**gamma #Gamma correction
    image = image*255

    # round and cast from float32 to uint8
    image = np.round(image)
    image = np.require(image, dtype=np.uint8)

    return image


In [ ]:
def my_deprocess_net_image(image, gamma):
    image = image.copy()              # don't modify destructively
    image = image[::-1]               # BGR -> RGB
    image = image.transpose(1, 2, 0)  # CHW -> HWC
    #image += [123, 117, 104]          # (approximately) undo mean subtraction

    # clamp values in [0, 255]
    image[image < 0], image[image > 255] = 0, 255
    
    image = image.astype(np.float32)
    image = image/255
    image = image**gamma #Gamma correction
    image = image*255

    # round and cast from float32 to uint8
    image = np.round(image)
    image = np.require(image, dtype=np.uint8)

    return image

In [ ]:
for i in range(5):
    img = my_deprocess_net_image(solver.net.blobs['data'].data[i+20], 0.5)
    plt.imshow(img)
    plt.figure(i+1)

In [ ]:
label = solver.net.blobs['illu'].data[20]
print label
print np.max(solver.net.blobs['data'].data[20])

In [ ]:
# We run the solver for niter times, and record the training loss.
for it in range(niter):
    solver.step(1)  # SGD by Caffe
    # store the train loss
    train_loss[it] = solver.net.blobs['loss'].data
    if it % 10 == 0:
        print 'iter %d, finetune_loss=%f, scratch_loss=' % (it, train_loss[it])
print 'done'